In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
# genes-pathways annotation

path = './KEGG_pathways/20230205_kegg_hsa.gmt'

files = open(path,encoding='utf-8')

files = files.readlines()

paways_genes_dict = {}
for i in files: 
    paways_genes_dict[i.split('\t')[0].split('_')[0]] = i.replace('\n','').split('\t')[2:] 


In [3]:
#mirna-pathways annotation
path = './KEGG_pathways/kegg_anano.txt'

files = open(path,encoding='utf-8')

files = files.readlines()

paways_mirna_dict = {}
for i in files:
     keys = i.split(',')[0].split('|')[1]
     values1 = i.split(',')[1:-1]
     values2 =  i.split(',')[-1].replace('\n','')
     values1.append(values2)
     values1 =list(set(values1)) 
     paways_mirna_dict[keys] = values1

In [4]:
union_kegg = list(set(paways_genes_dict.keys()).intersection(set(paways_mirna_dict.keys())))

In [5]:
paways_genes_dicts ={}
paways_mirna_dicts ={}

for i in union_kegg:
    paways_genes_dicts[i] = paways_genes_dict[i]
    
for i in union_kegg:
    paways_mirna_dicts[i] = paways_mirna_dict[i]    

In [6]:

genes_existed_pathway = []

mirna_existed_pathway = []

for index in paways_genes_dicts.keys():
    genes_existed_pathway = genes_existed_pathway+ list(paways_genes_dicts[index])
genes_existed_pathway = set(genes_existed_pathway)


for index in paways_mirna_dicts.keys():
    mirna_existed_pathway = mirna_existed_pathway+ list(paways_mirna_dicts[index])
mirna_existed_pathway = set(mirna_existed_pathway)



In [7]:
print(len(genes_existed_pathway))
print(len(mirna_existed_pathway))

7768
812


In [8]:
#Loading data
snv_data = pd.read_csv("./BLCA_data/snv_data.csv",index_col = 0)

miRNA_data = pd.read_csv("./BLCA_data/miRNA_data.csv",index_col = 0)

mRNA_data = pd.read_csv("./BLCA_data/mRNA_data.csv",index_col = 0)

example_case = pd.read_csv('./BLCA_data/response.csv',index_col=0)

In [9]:
print(snv_data.shape)
print(mRNA_data.shape)
print(miRNA_data.shape)

(402, 1000)
(402, 1000)
(402, 100)


In [10]:
union_gene_snv = list(snv_data.columns)
union_gene_miRNA = list(miRNA_data.columns)
union_gene_mRNA = list(mRNA_data.columns)

In [11]:
pathway_union = list(paways_genes_dicts.keys())
len(pathway_union)

238

In [12]:
mask_list = [union_gene_snv,union_gene_mRNA]

gene_pathway_bp_dfs = []


for i in range(len(mask_list)):
    pathways_genes = np.zeros((len(pathway_union), len(mask_list[i]))) 
    for p  in pathway_union:
        gs = paways_genes_dicts[p]
        g_inds = [mask_list[i].index(g) for g in gs if g in mask_list[i]]
        p_ind = pathway_union.index(p)
        pathways_genes[p_ind, g_inds] = 1
    gene_pathway_bp = pd.DataFrame(pathways_genes, index=pathway_union, columns=mask_list[i])
    
    gene_pathway_bp_dfs.append(gene_pathway_bp)
    

pathways_genes = np.zeros((len(pathway_union), len(union_gene_miRNA))) 
for p  in pathway_union:
    gs = paways_mirna_dicts[p]
    g_inds = [union_gene_miRNA.index(g) for g in gs if g in union_gene_miRNA]
    p_ind = pathway_union.index(p)
    pathways_genes[p_ind, g_inds] = 1
gene_pathway_bp = pd.DataFrame(pathways_genes, index=pathway_union, columns=union_gene_miRNA)

gene_pathway_bp_dfs.append(gene_pathway_bp)
    


In [20]:
# gene_pathway_bp_dfs

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.initializers import glorot_uniform, Initializer
from tensorflow.keras.layers import Input, Dense, Dropout, Embedding, GlobalAveragePooling1D,Layer
from tensorflow.keras import initializers,activations,regularizers
from tensorflow.keras.regularizers import Regularizer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.python.framework.ops import disable_eager_execution
 
tf.compat.v1.disable_eager_execution()

In [14]:
class Biological_module(Layer):
    def __init__(self, units, mapp=None, nonzero_ind=None, kernel_initializer='glorot_uniform', W_regularizer=None,
                 activation='tanh', use_bias=True,bias_initializer='zeros', bias_regularizer=None,
                 bias_constraint=None,**kwargs):
        
        self.units = units
        self.activation = activation
        self.mapp = mapp
        self.nonzero_ind = nonzero_ind
        self.use_bias = use_bias
        
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.kernel_regularizer = regularizers.get(W_regularizer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activation_fn = activations.get(activation)
        super(Biological_module, self).__init__(**kwargs)

        
    def build(self, input_shape):
        
        input_dim = input_shape[1]
   

        if not self.mapp is None:
            self.mapp = self.mapp.astype(np.float32)

   
        if self.nonzero_ind is None:
            nonzero_ind = np.array(np.nonzero(self.mapp)).T
            self.nonzero_ind = nonzero_ind

        self.kernel_shape = (input_dim, self.units)
        

        nonzero_count = self.nonzero_ind.shape[0]   


        self.kernel_vector = self.add_weight(name='kernel_vector',
                                             shape=(nonzero_count,),
                                             initializer=self.kernel_initializer,
                                             regularizer=self.kernel_regularizer,
                                             trainable=True)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer
                                        )
        else:
            self.bias = None

        super(Biological_module, self).build(input_shape)  
      

    def call(self, inputs):
        
        
        trans = tf.scatter_nd(tf.constant(self.nonzero_ind, tf.int32), self.kernel_vector,
                           tf.constant(list(self.kernel_shape)))
    
        output = K.dot(inputs, trans)
        
    
        if self.use_bias:
            output = K.bias_add(output, self.bias)
            
        if self.activation_fn is not None:
            output = self.activation_fn(output)

        return output

    def get_config(self):
        config = {
            'units': self.units,
            'activation': self.activation,
            'use_bias': self.use_bias,
            'nonzero_ind': np.array(self.nonzero_ind),
          
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),

            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'W_regularizer': regularizers.serialize(self.kernel_regularizer),

        }
        base_config = super(Biological_module, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
      
        return (input_shape[0], self.units)

In [15]:
class Self_Attention(Layer):
 
    def __init__(self, output_dim,  W_regularizer=None,**kwargs):
        self.output_dim = output_dim
        self.kernel_regularizer = regularizers.get(W_regularizer)
        super(Self_Attention, self).__init__(**kwargs)
 
    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(3,input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      regularizer=self.kernel_regularizer,
                                      trainable=True)
 
        super(Self_Attention, self).build(input_shape)  
 
    def call(self, x):
        WQ = K.dot(x, self.kernel[0])
        WK = K.dot(x, self.kernel[1])
        WV = K.dot(x, self.kernel[2])
 

        print("K.permute_dimensions(WK.shape",(K.permute_dimensions(WK,[1,0]).shape))
 
        QK =  K.dot(K.permute_dimensions(WK,[1,0]),WQ)
    
 
        QK = QK / (64**0.5)
 
        QK = K.softmax(QK)
 
        print("QK.shape",QK.shape)
 
        V = K.dot(WV,QK)
        
        print(V.shape)
 
        return V

    def get_config(self):
        config = {
          
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),


        }
        base_config = super(Self_Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
 
    def compute_output_shape(self, input_shape):
 
        return (input_shape[0],input_shape[1],self.output_dim)
 

In [16]:
def create_model(snv_data,mRNA_data,miRNA_data):
    
    S_inputs_snv = Input(shape=(snv_data.shape[1],), dtype='float32',name= 'snv_inputs')
    
    S_inputs_mRNA = Input(shape=(mRNA_data.shape[1],), dtype='float32',name= 'mRNA_inputs')
  
    S_inputs_miRNA = Input(shape=(miRNA_data.shape[1],), dtype='float32',name= 'miRNA_inputs')
    

    h0_snv = Biological_module(gene_pathway_bp_dfs[0].shape[0],mapp =gene_pathway_bp_dfs[0].values.T, name = 'h0_snv',W_regularizer=l2(0.001))(S_inputs_snv)

    
    h0_mRNA = Biological_module(gene_pathway_bp_dfs[1].shape[0],mapp =gene_pathway_bp_dfs[1].values.T, name = 'h0_mRNA',W_regularizer=l2(0.001))(S_inputs_mRNA)

    
    h0_miRNA = Biological_module(gene_pathway_bp_dfs[2].shape[0],mapp =gene_pathway_bp_dfs[2].values.T, name = 'h0_miRNA',W_regularizer=l2(0.001))(S_inputs_miRNA)


    atten1 = Self_Attention(64,W_regularizer=l2(0.001))(h0_snv)
    atten2 = Self_Attention(64,W_regularizer=l2(0.001))(h0_mRNA)
    atten3 = Self_Attention(64,W_regularizer=l2(0.001))(h0_miRNA)
    
    feature_tal = tf.keras.layers.concatenate([atten1,atten2,atten3])

    
    h4 = tf.keras.layers.Dense(32,activation='tanh')(feature_tal)
    
    h5 = tf.keras.layers.Dense(1,activation='sigmoid')(h4)
    

    model = Model(inputs=[S_inputs_snv,S_inputs_mRNA,S_inputs_miRNA], outputs=h5)

    model.summary()

    opt = tf.keras.optimizers.Adam(learning_rate = 0.0001,decay=0.0001) 
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['acc'])
    return model

In [24]:
#Evaluation function
import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
   

from sklearn.metrics import precision_recall_curve
def get_metrics(true_score,pre_score,pre_probe):
    
  
    fpr, tpr, thresholds = metrics.roc_curve(true_score, pre_probe, pos_label=1)
   
    auc = metrics.auc(fpr, tpr)
    
    aupr = average_precision_score(true_score, pre_probe)
    
    pre, rec, thresholds = precision_recall_curve(true_score, pre_probe)    
    auprc  = metrics.auc(rec, pre)
    
    
    accuracy = accuracy_score(true_score,pre_score)
    
    f1 = metrics.f1_score(true_score, pre_score)
    
    precision = metrics.precision_score(true_score,pre_score)
    
    recall = metrics.recall_score(true_score,pre_score)
    
     
    print( print(confusion_matrix(true_score,pre_score)))
    return precision,accuracy,recall,f1,auc,aupr,auprc


def evaluates(y_test, y_pred):
    
    auc = metrics.roc_auc_score(y_test,y_pred)
    
    aupr = average_precision_score(y_test, y_pred)
    
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)    
    auprc  = metrics.auc(recall, precision)
    
    pp = [1 if index>=0.5  else 0 for index in  y_pred ]
    
    pre = metrics.precision_score(y_test,pp)
    
    f1 = metrics.f1_score(y_test,pp)
    
    rec = metrics.recall_score(y_test,pp)
    
    acc = metrics.accuracy_score(y_test,pp)
    
    print(confusion_matrix(y_test,pp))
    return pre,acc,rec,f1,auc,aupr,auprc

In [18]:
snv_data = snv_data.loc[example_case.index]

miRNA_data = miRNA_data.loc[example_case.index]

mRNA_data = mRNA_data.loc[example_case.index]

example_case = example_case.loc[example_case.index]

y = example_case['response'].values

In [19]:

n_samples =example_case['response'].values

print(len(n_samples),n_samples.sum(),(len(n_samples) -n_samples.sum()))

x_0 =  len(n_samples) / (2*  (len(n_samples) -n_samples.sum()))
x_1 =  len(n_samples) / (2*  n_samples.sum())

print(x_0,x_1)

402 143 259
0.7760617760617761 1.4055944055944056


In [20]:
#feature importance
from keras.models import Sequential
def get_layers(model, level=1):
    layers = []
    for i, l in enumerate(model.layers):

        if type(l) == Sequential:
            layers.extend(get_layers(l, level + 1))
        else:
            layers.append(l)

    return layers



positive_example= example_case[example_case['response']==1]
snv_data_p = snv_data.loc[positive_example.index].values
mRNA_data_p  = mRNA_data.loc[positive_example.index].values
miRNA_data_p = miRNA_data.loc[positive_example.index].values


data_pd_s = [snv_data,mRNA_data,miRNA_data]
data_pd_name =['snv','mRNA','miRNA']
def get_weigjts(model,p):
    
   
    feed_dict ={}
    model_layers = get_layers(model)
    multi_data_inputs = [snv_data_p,mRNA_data_p,miRNA_data_p]
    for i in range(0,3):
        feed_dict[model.get_layer(model_layers[i].name).output] = multi_data_inputs[i]
        
    for k in range(0,3):
        meth_inputs_name = model_layers[k].name
        X = model.get_layer(meth_inputs_name).output 
        Y  = model.outputs[-1]
        ret = [g * (x - b) for g, x, b in zip(tf.gradients(Y, X), [X], [np.zeros((multi_data_inputs[k].shape))[0].shape])]
        sess = tf.compat.v1.InteractiveSession()
        sess.run(tf.compat.v1.initialize_all_variables())
        evaluated_gradients = sess.run(ret,feed_dict=feed_dict)
        gene_pd = pd.DataFrame(columns=['genes','values'])
        gene_pd['genes'] = data_pd_s[k].columns
        gene_pd['values'] = evaluated_gradients[0].sum(axis=0)
        gene_pd.to_csv('./coef_weight/BLCA/h{}/{}.csv'.format(p,data_pd_name[k]))

In [26]:
#Five-fold cross validation
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1029) 

kfscore = []
p = 0
for train_index, test_index in skf.split(snv_data.values,y):

    snv_train_x = snv_data.values[train_index]
    snv_test_x  = snv_data.values[test_index]

    mRNA_train_x = mRNA_data.values[train_index]
    mRNA_test_x  = mRNA_data.values[test_index]

    miRNA_train_x = miRNA_data.values[train_index]
    miRNA_test_x  = miRNA_data.values[test_index]

    train_y  = y[train_index]
    test_y   = y[test_index]

    model = create_model(snv_data,mRNA_data,miRNA_data)
    model.fit( {"snv_inputs": snv_train_x,  "mRNA_inputs": mRNA_train_x, 'miRNA_inputs':miRNA_train_x},train_y,
                 validation_data=({"snv_inputs": snv_test_x, "mRNA_inputs": mRNA_test_x,'miRNA_inputs':miRNA_test_x},test_y),
                 epochs=170,batch_size = 64,class_weight = {0:x_0,1:x_1})  

    y_pred = model.predict({"snv_inputs": snv_test_x, "mRNA_inputs": mRNA_test_x,'miRNA_inputs':miRNA_test_x})

    y_score = [1 if index>=0.5  else 0 for index in  y_pred]

    evaluate_epoch = get_metrics(test_y,y_score,y_pred)
    print(evaluate_epoch)
#     get_weigjts(model,p)
#     p = p + 1
    kfscore.append(evaluate_epoch)
    
results = list(np.array(kfscore).sum(axis= 0)/5.0)
print('Cross validated results :  ACC = {}, REC = {}, F1 = {}, AUC = {}, AUPR ={}'.format(results[1],results[2],results[3],results[4],results[5]))

K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 snv_inputs (InputLayer)        [(None, 1000)]       0           []                               
                                                                                                  
 mRNA_inputs (InputLayer)       [(None, 1000)]       0           []                               
                                                                                                  
 miRNA_inputs (InputLayer)      [(None, 100)]        0           []                               
                                                                                

D:\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


321/321 [==============================] - 1s 2ms/sample - loss: 0.8090 - acc: 0.4798 - val_loss: 0.8069 - val_acc: 0.3704
Epoch 2/170
321/321 [==============================] - 0s 423us/sample - loss: 0.8049 - acc: 0.4579 - val_loss: 0.8026 - val_acc: 0.6543
Epoch 3/170
321/321 [==============================] - 0s 590us/sample - loss: 0.8012 - acc: 0.6511 - val_loss: 0.7985 - val_acc: 0.6420
Epoch 4/170
321/321 [==============================] - 0s 475us/sample - loss: 0.7977 - acc: 0.6542 - val_loss: 0.7955 - val_acc: 0.5556
Epoch 5/170
321/321 [==============================] - 0s 298us/sample - loss: 0.7941 - acc: 0.5296 - val_loss: 0.7930 - val_acc: 0.3580
Epoch 6/170
321/321 [==============================] - 0s 295us/sample - loss: 0.7908 - acc: 0.3551 - val_loss: 0.7907 - val_acc: 0.3580
Epoch 7/170
321/321 [==============================] - 0s 329us/sample - loss: 0.7874 - acc: 0.3551 - val_loss: 0.7876 - val_acc: 0.3580
Epoch 8/170
321/321 [==============================] - 

321/321 [==============================] - 0s 407us/sample - loss: 0.2307 - acc: 0.9315 - val_loss: 0.3790 - val_acc: 0.8642
Epoch 120/170
321/321 [==============================] - 0s 490us/sample - loss: 0.2279 - acc: 0.9377 - val_loss: 0.3704 - val_acc: 0.8642
Epoch 121/170
321/321 [==============================] - 0s 410us/sample - loss: 0.2235 - acc: 0.9408 - val_loss: 0.3601 - val_acc: 0.8889
Epoch 122/170
321/321 [==============================] - 0s 427us/sample - loss: 0.2208 - acc: 0.9439 - val_loss: 0.3528 - val_acc: 0.8889
Epoch 123/170
321/321 [==============================] - 0s 510us/sample - loss: 0.2186 - acc: 0.9470 - val_loss: 0.3489 - val_acc: 0.8889
Epoch 124/170
321/321 [==============================] - 0s 613us/sample - loss: 0.2167 - acc: 0.9502 - val_loss: 0.3466 - val_acc: 0.8889
Epoch 125/170
321/321 [==============================] - 0s 601us/sample - loss: 0.2147 - acc: 0.9502 - val_loss: 0.3468 - val_acc: 0.8889
Epoch 126/170
321/321 [==================

D:\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


...

D:\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


322/322 [==============================] - 1s 2ms/sample - loss: 0.8081 - acc: 0.4348 - val_loss: 0.8064 - val_acc: 0.4250
Epoch 2/170
322/322 [==============================] - 0s 279us/sample - loss: 0.8040 - acc: 0.3789 - val_loss: 0.8026 - val_acc: 0.3875
Epoch 3/170
322/322 [==============================] - 0s 338us/sample - loss: 0.8001 - acc: 0.4161 - val_loss: 0.7984 - val_acc: 0.5875
Epoch 4/170
322/322 [==============================] - 0s 251us/sample - loss: 0.7963 - acc: 0.6584 - val_loss: 0.7942 - val_acc: 0.6375
Epoch 5/170
322/322 [==============================] - 0s 254us/sample - loss: 0.7926 - acc: 0.6522 - val_loss: 0.7900 - val_acc: 0.6375
Epoch 6/170
322/322 [==============================] - 0s 282us/sample - loss: 0.7890 - acc: 0.6460 - val_loss: 0.7859 - val_acc: 0.6375
Epoch 7/170
322/322 [==============================] - 0s 406us/sample - loss: 0.7855 - acc: 0.6460 - val_loss: 0.7819 - val_acc: 0.6375
Epoch 8/170
322/322 [==============================] - 

322/322 [==============================] - 0s 251us/sample - loss: 0.1892 - acc: 0.9503 - val_loss: 0.3194 - val_acc: 0.8750
Epoch 120/170
322/322 [==============================] - 0s 248us/sample - loss: 0.1847 - acc: 0.9596 - val_loss: 0.3145 - val_acc: 0.9125
Epoch 121/170
322/322 [==============================] - 0s 282us/sample - loss: 0.1822 - acc: 0.9596 - val_loss: 0.3125 - val_acc: 0.9000
Epoch 122/170
322/322 [==============================] - 0s 257us/sample - loss: 0.1802 - acc: 0.9596 - val_loss: 0.3115 - val_acc: 0.9125
Epoch 123/170
322/322 [==============================] - 0s 260us/sample - loss: 0.1784 - acc: 0.9596 - val_loss: 0.3108 - val_acc: 0.9000
Epoch 124/170
322/322 [==============================] - 0s 288us/sample - loss: 0.1767 - acc: 0.9596 - val_loss: 0.3087 - val_acc: 0.9125
Epoch 125/170
322/322 [==============================] - 0s 273us/sample - loss: 0.1754 - acc: 0.9596 - val_loss: 0.3085 - val_acc: 0.9000
Epoch 126/170
322/322 [==================

D:\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


[[47  4]
 [ 6 23]]
None
(0.8518518518518519, 0.875, 0.7931034482758621, 0.8214285714285715, 0.9716024340770791, 0.9534158462540174, 0.9526103172063547)
Cross validated results :  ACC = 0.9004938271604939, REC = 0.90935960591133, F1 = 0.8661303637403724, AUC = 0.9657395849586518, AUPR =0.9531595736064723


In [ ]:
#feature importance
#The average results of the five-fold crossover experiments

import pandas as pd
dirs_paths = './coef_weight/BLCA'
dirs_genes = ['snv.csv','mRNA.csv','miRNA.csv']
dirs = ['h0','h1','h2','h3','h4']

for i in range(0,3):
    t_gene_data = pd.DataFrame()
    for j in dirs:
        print(dirs_paths+'\\'+j+'\\'+dirs_genes[i])
        genes_data = pd.read_csv(dirs_paths+'\\'+j+'\\'+dirs_genes[i])
        t_gene_data = t_gene_data.append(genes_data)
    temp_gene = t_gene_data.groupby('genes').mean().sort_values('values',ascending=False)
    temp_gene.to_csv(dirs_paths+'\\average\\average_'+dirs_genes[i])

In [29]:

def create_single_model(inputs,omics_id):
   
    
    S_inputs = Input(shape=(inputs.shape[1],), dtype='float32')
    


    h0 = Biological_module(gene_pathway_bp_dfs[omics_id].shape[0],mapp =gene_pathway_bp_dfs[omics_id].values.T,W_regularizer=l2(0.001))(S_inputs)
    

    atten1 = Self_Attention(64,W_regularizer=l2(0.001))(h0)
    
    h4 = tf.keras.layers.Dense(32,activation='tanh')(atten1)
    
    h5 = tf.keras.layers.Dense(1,activation='sigmoid')(h4)

    model = Model(inputs=S_inputs, outputs=h5)

    model.summary()

    opt = tf.keras.optimizers.Adam(learning_rate = 0.0001,decay=0.0001) #,decay=-0.0001
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['acc'])
    return model

In [31]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=1029)

kfscore = []
for train_index, test_index in skf.split(snv_data.values,y):

    single_train_x = snv_data.values[train_index]
    single_test_x  = snv_data.values[test_index]

    train_y  = y[train_index]
    test_y   = y[test_index]


    single_model = create_single_model(single_train_x,0)


    single_model.fit(single_train_x,train_y,validation_data=(single_test_x,test_y),
                      epochs=170,batch_size = 64,class_weight = {0:x_0,1:x_1}) 

    y_pred = single_model.predict(single_test_x)

    evaluate_epoch = evaluates(test_y,y_pred)
    print(evaluate_epoch)
    kfscore.append(evaluate_epoch)
    
results = list(np.array(kfscore).sum(axis= 0)/5.0)   
print('Cross validated results :  ACC = {}, REC = {}, F1 = {}, AUC = {}, AUPR ={}'.format(results[1],results[2],results[3],results[4],results[5]))



WQ.shape (None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1000)]            0         
                                                                 
 biological_module (Biologic  (None, 238)              3170      
 al_module)                                                      
                                                                 
 self__attention_18 (Self_At  (None, 64)               45696     
 tention)                                                        
                                                                 
 dense_12 (Dense)            (None, 32)                2080      
                                                                 
 dense_13 (Dense)            (None, 1)                 33        
                                   

6/6 [==============================] - 0s 22ms/step - loss: 0.6000 - acc: 0.9034 - val_loss: 0.6183 - val_acc: 0.8519
Epoch 52/170
6/6 [==============================] - 0s 17ms/step - loss: 0.5920 - acc: 0.8847 - val_loss: 0.6160 - val_acc: 0.8025
Epoch 53/170
6/6 [==============================] - 0s 14ms/step - loss: 0.5840 - acc: 0.8629 - val_loss: 0.6143 - val_acc: 0.7778
Epoch 54/170
6/6 [==============================] - 0s 11ms/step - loss: 0.5761 - acc: 0.8505 - val_loss: 0.6105 - val_acc: 0.7901
Epoch 55/170
6/6 [==============================] - 0s 12ms/step - loss: 0.5687 - acc: 0.8349 - val_loss: 0.6078 - val_acc: 0.7407
Epoch 56/170
6/6 [==============================] - 0s 20ms/step - loss: 0.5609 - acc: 0.8224 - val_loss: 0.6022 - val_acc: 0.7407
Epoch 57/170
6/6 [==============================] - 0s 19ms/step - loss: 0.5539 - acc: 0.8380 - val_loss: 0.5954 - val_acc: 0.7901
Epoch 58/170
6/6 [==============================] - 0s 19ms/step - loss: 0.5464 - acc: 0.8505 - 

[[45  7]
 [ 3 26]]
(0.7878787878787878, 0.8765432098765432, 0.896551724137931, 0.8387096774193549, 0.9565649867374004, 0.936763460705649, 0.9375869581223697)
WQ.shape (None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1000)]            0         
                                                                 
 biological_module_1 (Biolog  (None, 238)              3170      
 ical_module)                                                    
                                                                 
 self__attention_19 (Self_At  (None, 64)               45696     
 tention)                                                        
                                                                 
 dense_14 (Dense)            (None, 32)                2080      
         

Epoch 50/170
6/6 [==============================] - 0s 12ms/step - loss: 0.5893 - acc: 0.7726 - val_loss: 0.6081 - val_acc: 0.7284
Epoch 51/170
6/6 [==============================] - 0s 11ms/step - loss: 0.5818 - acc: 0.8006 - val_loss: 0.6002 - val_acc: 0.7407
Epoch 52/170
6/6 [==============================] - 0s 11ms/step - loss: 0.5742 - acc: 0.8162 - val_loss: 0.5924 - val_acc: 0.7531
Epoch 53/170
6/6 [==============================] - 0s 13ms/step - loss: 0.5660 - acc: 0.8287 - val_loss: 0.5876 - val_acc: 0.7407
Epoch 54/170
6/6 [==============================] - 0s 12ms/step - loss: 0.5578 - acc: 0.8131 - val_loss: 0.5838 - val_acc: 0.7407
Epoch 55/170
6/6 [==============================] - 0s 11ms/step - loss: 0.5497 - acc: 0.8037 - val_loss: 0.5782 - val_acc: 0.7407
Epoch 56/170
6/6 [==============================] - 0s 16ms/step - loss: 0.5415 - acc: 0.7913 - val_loss: 0.5740 - val_acc: 0.7407
Epoch 57/170
6/6 [==============================] - 0s 15ms/step - loss: 0.5330 - a

 input_3 (InputLayer)        [(None, 1000)]            0         
                                                                 
 biological_module_2 (Biolog  (None, 238)              3170      
 ical_module)                                                    
                                                                 
 self__attention_20 (Self_At  (None, 64)               45696     
 tention)                                                        
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 1)                 33        
                                                                 
Total params: 50,979
Trainable params: 50,979
Non-trainable params: 0
_________________________________________________________________
Epoch 1/170
WQ.shape (None, 64)
K.permute_dimensions(WK.shape (64, None)

Epoch 53/170
6/6 [==============================] - 0s 12ms/step - loss: 0.5224 - acc: 0.8230 - val_loss: 0.5457 - val_acc: 0.7375
Epoch 54/170
6/6 [==============================] - 0s 14ms/step - loss: 0.5137 - acc: 0.8106 - val_loss: 0.5418 - val_acc: 0.7250
Epoch 55/170
6/6 [==============================] - 0s 12ms/step - loss: 0.5060 - acc: 0.8168 - val_loss: 0.5334 - val_acc: 0.7500
Epoch 56/170
6/6 [==============================] - 0s 13ms/step - loss: 0.4976 - acc: 0.8230 - val_loss: 0.5273 - val_acc: 0.7500
Epoch 57/170
6/6 [==============================] - 0s 13ms/step - loss: 0.4894 - acc: 0.8385 - val_loss: 0.5183 - val_acc: 0.8000
Epoch 58/170
6/6 [==============================] - 0s 12ms/step - loss: 0.4815 - acc: 0.8509 - val_loss: 0.5120 - val_acc: 0.8000
Epoch 59/170
6/6 [==============================] - 0s 12ms/step - loss: 0.4743 - acc: 0.8634 - val_loss: 0.5035 - val_acc: 0.8250
Epoch 60/170
6/6 [==============================] - 0s 13ms/step - loss: 0.4667 - a

 self__attention_21 (Self_At  (None, 64)               45696     
 tention)                                                        
                                                                 
 dense_18 (Dense)            (None, 32)                2080      
                                                                 
 dense_19 (Dense)            (None, 1)                 33        
                                                                 
Total params: 50,979
Trainable params: 50,979
Non-trainable params: 0
_________________________________________________________________
Epoch 1/170
WQ.shape (None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
WQ.shape (None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
1/6 [====>.........................] - ETA: 3s - loss: 0.7067 - acc: 0.4219WQ.shape (None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
6/6 [==============================] - 1s 49m

Epoch 56/170
6/6 [==============================] - 0s 11ms/step - loss: 0.4994 - acc: 0.7888 - val_loss: 0.5458 - val_acc: 0.7250
Epoch 57/170
6/6 [==============================] - 0s 14ms/step - loss: 0.4906 - acc: 0.8292 - val_loss: 0.5355 - val_acc: 0.7375
Epoch 58/170
6/6 [==============================] - 0s 11ms/step - loss: 0.4820 - acc: 0.8571 - val_loss: 0.5274 - val_acc: 0.7500
Epoch 59/170
6/6 [==============================] - 0s 11ms/step - loss: 0.4741 - acc: 0.8696 - val_loss: 0.5180 - val_acc: 0.7875
Epoch 60/170
6/6 [==============================] - 0s 11ms/step - loss: 0.4669 - acc: 0.8851 - val_loss: 0.5133 - val_acc: 0.7750
Epoch 61/170
6/6 [==============================] - 0s 13ms/step - loss: 0.4590 - acc: 0.8727 - val_loss: 0.5142 - val_acc: 0.7375
Epoch 62/170
6/6 [==============================] - 0s 11ms/step - loss: 0.4518 - acc: 0.8571 - val_loss: 0.5126 - val_acc: 0.7375
Epoch 63/170
6/6 [==============================] - 0s 11ms/step - loss: 0.4448 - a

                                                                 
Total params: 50,979
Trainable params: 50,979
Non-trainable params: 0
_________________________________________________________________
Epoch 1/170
WQ.shape (None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
WQ.shape (None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
1/6 [====>.........................] - ETA: 3s - loss: 0.7543 - acc: 0.6094WQ.shape (None, 64)
K.permute_dimensions(WK.shape (64, None)
QK.shape (64, 64)
(None, 64)
6/6 [==============================] - 1s 50ms/step - loss: 0.7311 - acc: 0.4565 - val_loss: 0.7310 - val_acc: 0.4250
Epoch 2/170
6/6 [==============================] - 0s 12ms/step - loss: 0.7298 - acc: 0.3882 - val_loss: 0.7299 - val_acc: 0.3875
Epoch 3/170
6/6 [==============================] - 0s 12ms/step - loss: 0.7284 - acc: 0.3540 - val_loss: 0.7287 - val_acc: 0.3750
Epoch 4/170
6/6 [==============================] - 0s 11ms/step

Epoch 59/170
6/6 [==============================] - 0s 11ms/step - loss: 0.4536 - acc: 0.8789 - val_loss: 0.5225 - val_acc: 0.8250
Epoch 60/170
6/6 [==============================] - 0s 12ms/step - loss: 0.4449 - acc: 0.8478 - val_loss: 0.5229 - val_acc: 0.8125
Epoch 61/170
6/6 [==============================] - 0s 11ms/step - loss: 0.4382 - acc: 0.8323 - val_loss: 0.5204 - val_acc: 0.8125
Epoch 62/170
6/6 [==============================] - 0s 13ms/step - loss: 0.4321 - acc: 0.8261 - val_loss: 0.5165 - val_acc: 0.8125
Epoch 63/170
6/6 [==============================] - 0s 12ms/step - loss: 0.4246 - acc: 0.8292 - val_loss: 0.5071 - val_acc: 0.8125
Epoch 64/170
6/6 [==============================] - 0s 12ms/step - loss: 0.4169 - acc: 0.8447 - val_loss: 0.4955 - val_acc: 0.8375
Epoch 65/170
6/6 [==============================] - 0s 12ms/step - loss: 0.4099 - acc: 0.8789 - val_loss: 0.4872 - val_acc: 0.8375
Epoch 66/170
6/6 [==============================] - 0s 12ms/step - loss: 0.4036 - a